<a href="https://colab.research.google.com/github/AInisha/hello-world/blob/main/hr_payment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
train= pd.read_csv('/content/drive/MyDrive/hr/trainhr.csv', parse_dates=['MMM-YY', 'Dateofjoining', 'LastWorkingDate'],dayfirst=True)


In [2]:
train.head(5)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1


In [3]:
test= pd.read_csv('/content/drive/MyDrive/hr/testhr.csv')
train1=train.copy()
# https://datascience.stackexchange.com/questions/39317/difference-between-ordinalencoder-and-labelencoder
# to identify designation by algo
enc= {'Bachelor':1, 'College':1, 'Master':2}
train1['Education_Level']= train1['Education_Level'].map(enc)
enc1= {'Male':1, 'Female':0}
train1['Gender']= train1['Gender'].map(enc1)
mean_data= train.groupby('Emp_ID')['Salary','Total Business Value', 'Quarterly Rating'].mean()

# average rating and average business value, but in my previous doc i made hike column
# dummy= pd.merge(train1, mean_data, how='left', on='Emp_ID')
#promotion column with 0 and 1 
# how many times promotion
# one= pd.DataFrame([1,2,2,2,2,2,2,2])
# two= pd.DataFrame([2,2,2,2,2,2,2,3])
# two.max()-one.mean()
# 0-->no promotion
# 1--> single promtion
# more than 1 --> more than 1 promotion

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if __name__ == '__main__':


In [4]:
#216, 139, 6,47 got high promotion
#1973 no promotion
designationdata1= pd.concat([train.groupby('Emp_ID')['Designation'].max(), train.groupby('Emp_ID')['Joining Designation'].mean()], axis=1)
designationdata1['promotion']=designationdata1['Designation']-designationdata1['Joining Designation']
dummy= pd.concat([mean_data, designationdata1], axis=1)
#number of working years/months and hike percentage
empid= []
hike=[]
for name, group in train.groupby('Emp_ID'):
  # print(group['Salary'].iloc[-1]-group['Salary'].iloc[0]))
  group1= group.reset_index()
  # if name== 4:
  #   print((group1['Salary'].iloc[-1]-group1['Salary'].iloc[0])/group1['Salary'].iloc[0]*100)
  abcd= (group1['Salary'].iloc[-1]-group1['Salary'].iloc[0])/group1['Salary'].iloc[0]*100
  hike.append(abcd)
  empid.append(name)
hike= pd.DataFrame({'Emp_ID': empid, 'hike': hike})
hike.set_index('Emp_ID', inplace= True)
dummy2=pd.merge(dummy, hike, how='left', on='Emp_ID')
cat= pd.concat([train1.groupby('Emp_ID')['Age'].max(), train1.groupby('Emp_ID')['Gender'].mean(), train1.groupby('Emp_ID')['Education_Level'].mean()], axis=1)
dummy3=pd.merge(dummy2, cat, how='left', on='Emp_ID')
dummy3.head()

,Salary,Total Business Value,Quarterly Rating,Designation,Joining Designation,promotion,hike,Age,Gender,Education_Level
Emp_ID,,,,,,,,,,
1,57387.0,571860.0,2.0,1,1.0,0.0,0.0,28,1.0,2.0
2,67016.0,0.0,1.0,2,2.0,0.0,0.0,31,1.0,2.0
4,65603.0,70000.0,1.0,2,2.0,0.0,0.0,43,1.0,2.0
5,46368.0,40120.0,1.0,1,1.0,0.0,0.0,29,1.0,1.0
6,78728.0,253000.0,1.6,3,3.0,0.0,0.0,31,0.0,1.0


In [5]:
id=[]
iddata=train1.groupby(['Emp_ID'])['City'].unique()
for i in range(2381):
  id.append(iddata.iloc[i][0])
iddata= pd.DataFrame(iddata)
iddata['City1']=id
dummy4=pd.merge(dummy3, iddata[['City1']], how='left', on='Emp_ID')
dummy4.head()

,Salary,Total Business Value,Quarterly Rating,Designation,Joining Designation,promotion,hike,Age,Gender,Education_Level,City1
Emp_ID,,,,,,,,,,,
1,57387.0,571860.0,2.0,1,1.0,0.0,0.0,28,1.0,2.0,C23
2,67016.0,0.0,1.0,2,2.0,0.0,0.0,31,1.0,2.0,C7
4,65603.0,70000.0,1.0,2,2.0,0.0,0.0,43,1.0,2.0,C13
5,46368.0,40120.0,1.0,1,1.0,0.0,0.0,29,1.0,1.0,C9
6,78728.0,253000.0,1.6,3,3.0,0.0,0.0,31,0.0,1.0,C11


In [6]:
#employee left org or not---> target
newcol= ['Emp_ID', 'LastWorkingDate']
df2= train1[newcol]
df2['Target']= df2['LastWorkingDate'].isnull()
df2['Target']= df2['Target'].apply(lambda x: 0 if x == True else 1)
ndf2= pd.DataFrame(df2.groupby('Emp_ID')['Target'].sum())
dummy5=pd.merge(dummy4, ndf2, how='left', on='Emp_ID')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [7]:
#year
#tenurity--> how many years he/she workedtenure= train1.groupby(['Emp_ID'])
a2= []
b2= []
c2= []
d2= []
e2= []
k5= train1.groupby('Emp_ID') 
for name, group in k5:
  group2= group.reset_index()
  if len(set(group2['LastWorkingDate'].unique()))>1:
    daysv= group2.iloc[-1, 9] - group2.iloc[-1, 8]
    a2.append(daysv.days)

  #resolve the issue when person has left within a month put if condition
  elif len(set(group2['LastWorkingDate'].unique()))==1:
    daysv1= group2.iloc[-1, 1] - group2.iloc[0, 1]
    a2.append(daysv1.days)
  
  c2.append(group2.iloc[-1,1].year)
  d2.append(group2.iloc[0,1])
  b2.append(name)
  e2.append(group2.iloc[-1, 9])


k6= pd.DataFrame(columns=['Emp_ID', 'tenurity','current_year', 'reporting_date', 'last_working_date'])
k6['Emp_ID']= b2
k6['tenurity']= a2
k6['tenurity']= k6['tenurity'].apply(lambda x: np.round(x/30, 1))
k6['current_year']= c2
k6['reporting_date']= d2
k6['last_working_date']= e2
k6.set_index('Emp_ID', inplace= True)
k6.head()


,tenurity,current_year,reporting_date,last_working_date
Emp_ID,,,,
1,2.6,2016,2016-01-01,2016-03-11
2,1.0,2017,2017-11-01,NaT
4,4.7,2017,2016-12-01,2017-04-27
5,1.9,2016,2016-01-01,2016-03-07
6,4.1,2017,2017-08-01,NaT


In [8]:
dummy6=pd.merge(dummy5, k6, how='left', on='Emp_ID')
dummy6.to_csv('/content/drive/MyDrive/hr/dummy6.csv')

In [9]:
# train and test, standarize, gradient and Xgboost for 2018
#people who have not left still can they live in the year 2018
#people who have not left still can they live in the year 2018#not left/ but whether they will leave within 2 quarters
dummy6[dummy6['Target']== 0].shape

(765, 16)

In [10]:
#left
dummy6[dummy6['Target']== 1].shape

(1616, 16)

In [ ]:
#challenge--> i donot have 2018 data of employee who has not left
#employee who has left after 2 quarters
# redo--> every quarter datset
#assuming there is no label and all the existing column will help in prediction, 
#column name would be person who has left in next 2 quarter in dataframe dummy6[dummy6['Target']== 1].shape
#In train have to prepare  a column where the person left in next 2 quarter

In [ ]:
#0---> left within same quarter
#2--> left wihin 2 quarter
newtrain= dummy6[dummy6['Target']== 1]
newtest=dummy6[dummy6['Target']== 0]
newtrain['differenceinqtr']=newtrain['reporting_date'].dt.quarter-newtrain['last_working_date'].dt.quarter
newtrain['left_in_2_qtr2']=newtrain['differenceinqtr'].map({-3: 0, -2:1, -1:1,  0:0,  1:1,  2:1,  3:0})

In [12]:
newtrain.head()

,Salary,Total Business Value,Quarterly Rating,Designation,Joining Designation,promotion,hike,Age,Gender,Education_Level,City1,Target,tenurity,current_year,reporting_date,last_working_date,differenceinqtr,left_in_2_qtr2
Emp_ID,,,,,,,,,,,,,,,,,,
1,57387.0,571860.0,2.0,1,1.0,0.0,0.0,28,1.0,2.0,C23,1,2.6,2016,2016-01-01,2016-03-11,0,0
4,65603.0,70000.0,1.0,2,2.0,0.0,0.0,43,1.0,2.0,C13,1,4.7,2017,2016-12-01,2017-04-27,2,1
5,46368.0,40120.0,1.0,1,1.0,0.0,0.0,29,1.0,1.0,C9,1,1.9,2016,2016-01-01,2016-03-07,0,0
8,70656.0,0.0,1.0,3,3.0,0.0,0.0,34,1.0,1.0,C2,1,1.9,2017,2017-09-01,2017-11-15,-1,1
12,28116.0,434530.0,2.5,1,1.0,0.0,0.0,35,1.0,2.0,C23,1,5.8,2016,2016-07-01,2016-12-21,-1,1


In [ ]:
# Certain machine learning algorithms such as distance based algorithms , curve based algorithms or matrix factorization, decomposition or dimensionality reduction or gradient descent based algorithms are sensitive towards feature scaling (standardization and normalization for numerical variables).

In [21]:
newtrainx= newtrain[['Salary',
 'Total Business Value',
 'Quarterly Rating',
 'Designation',
 'Joining Designation',
 'promotion',
 'hike',
 'Age',
 'Gender',
 'Education_Level',
 'Target',
 'tenurity']]

newtrainy= newtrain[['left_in_2_qtr2']]

newtestx= newtest[['Salary',
 'Total Business Value',
 'Quarterly Rating',
 'Designation',
 'Joining Designation',
 'promotion',
 'hike',
 'Age',
 'Gender',
 'Education_Level',
 'Target',
 'tenurity']]

In [23]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score
ada= AdaBoostClassifier()
ada.fit(newtrainx, newtrainy)
pred= ada.predict(newtrainx)
f1_score(newtrainy, pred)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.803096274794388

In [26]:
testpred= ada.predict(newtestx)

In [28]:
newtestx['left_in_2_qtr2']= list(testpred)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
newtestx.to_csv('/content/drive/MyDrive/hr/finaltest.csv')